In [ ]:
# create object with parameters
# run simulation, have the results stored in a df, also have a summary report
# visualize the results

In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime 
import numpy as np

In [2]:
url = 'http://agebb.missouri.edu/weather/history/report.asp?station_prefix=bwk&start_month=1&end_month=2&start_day=1&end_day=15&start_year=2020&end_year=2020&period_type=1&convert=1&field_elements=70&field_elements=3&field_elements=51&field_elements=73'
headers = {'User-Agent':'Mozilla/5.0'}
response = requests.get(url, headers = headers)

In [3]:
soup = BeautifulSoup(response.content, 'html.parser')
table = soup.find('pre')
data = table.contents[0].split()

In [4]:
column_names = ['Date', 'Total Precip', 'Max Temp', 'Min Temp', 'Total Solar Rad']
weather_data = pd.DataFrame(columns=column_names)

In [5]:
i = 18
while i < len(data) :
    newrow = data[i:i+7]    
    datestring = newrow[0] + '-' + newrow[1] + '-' + newrow[2]
    dateobj = datetime.datetime.strptime(datestring, '%m-%d-%Y').date()
    weather_data = weather_data.append({'Date': dateobj, 
                    'Total Precip': newrow[3], 
                    'Max Temp': newrow[4],
                    'Min Temp': newrow[5],
                    'Total Solar Rad': newrow[6]}, ignore_index=True)
    i += 7
weather_data['Date'] = pd.to_datetime(weather_data['Date'])
weather_data.set_index('Date', inplace = True)

In [47]:
crop_params = {
    'N stuff': 25,
    'k': 0.6,
    'LAI by stage':[[0, .003], [.5, .009], [1, .006], [1.5, .004], [2, .001]],
    'Max Increase of Roots per day': .012, # m d-1
    'Partitioning %s Roots': [
    [0.0, 0.60],
    [0.33, 0.58],
    [0.40, 0.55],
    [0.80, 0.10],
    [1.00, 0.00],
    [2.00, 0.0 ]],

    'Partitioning %s Leaves': [
    [0.0, 0.40],
    [0.33, 0.42],
    [0.40, 0.405],
    [0.80, 0.36],
    [1.00, 0.10],
    [1.01, 0.00],
    [2.00, 0.00]],

    'Partitioning %s Stem': [
    [0.0, 0.00],
    [0.33, 0.00],
    [0.40, 0.045],
    [0.80, 0.54],
    [1.00, 0.90],
    [1.01, 0.25],
    [2.00, 0.00]],

    'Partitioning %s Storage Organs': [
    [0.0, 0.00],
    [0.33, 0.00],
    [0.40, 0.00],
    [0.80, 0.00],
    [1.00, 0.00],
    [1.01, 0.75],
    [2.00, 1.00]],
    
    'RUE': 2.8, # g / MJ ?
    'Temp Sum for Emergence': 800,
    'Temp Sum for Anthesis': 300,
    'Temp Sum for Maturity': 600,
    'Planting Date': datetime.date(2020, 1, 4)
}

In [45]:
soil_params = {
    'Initial N': 200,
    'Initial SM': .3,
    'Water Retention': [[.3, 2],[.4, 3]], # step function of SM and pF
    'SM @ WP': .09,
    'SM @ FC': .3,
    'SM @ Sat': .5
}

In [ ]:
weather_data = pd.DataFrame()

In [15]:
management_params = {
    'Amount N': 20,
    'pF to Maintain': 2.2
}

In [57]:
class Model :
    def __init__(self, crop, soil, weather, management) :
        # read variables 
        self.N = crop['N stuff']
        self.k = crop['k']
        self.LAIRates = pd.DataFrame(crop['LAI by stage'])
        self.RGMAX = crop['Max Increase of Roots per day']
        self.RT = pd.DataFrame(crop['Partitioning %s Roots'])
        self.LV = pd.DataFrame(crop['Partitioning %s Leaves'])
        self.ST = pd.DataFrame(crop['Partitioning %s Stem'])
        self.SO = pd.DataFrame(crop['Partitioning %s Storage Organs'])
        self.RUE = crop['RUE']
        self.TSUMEM = crop['Temp Sum for Emergence']
        self.TSUM1 = crop['Temp Sum for Anthesis']
        self.TSUM2 = crop['Temp Sum for Maturity']
        self.DOS = crop['Planting Date'] # date of sowing
        self.today = crop['Planting Date']
        self.LAI = .5
        
        self.IN = soil['Initial N']
        self.ISM = soil['Initial SM']
        self.WRC = pd.DataFrame(soil['Water Retention']) # Water Retention Curve
        self.WPSM = soil['SM @ WP']
        self.FCSM = soil['SM @ FC']
        self.SSM = soil['SM @ Sat']
        
        self.weather = weather
        
        self.management = management
        
        
    def update() :
        self.today += timedelta(days=1)
        #update partitioning proportions
    
    def radiation_intercepted() :
        radiation = float(self.weather.loc[self.today]['Total Solar Rad']) 
        ri = radiation * (1 - np.exp(-1 * self.k * self.LAI))
        return ri
    
    def run() :
        
        potential_biomass = radiation_intercepted() * self.RUE
        # actual biomass limited by water and nitrogen availability
        ##### 
        # then partition it
        self.roots += biomass * self.root_partition
        self.leaves += biomass * self.leaves_partition
        # something here happens to the LAI ######
        self.stem += biomass * self.stem_partition
        self.storage += biomass * self.storage_partition
        

In [58]:
model = Model(crop_params, soil_params, weather_data, management_params)

In [ ]:
# Nitrogen - apply a standard amount at first and then apply a set amount to an area when it's Nitrogen starved
# Irrigation - maintain a certain pF